# Astrophysics Reviewer Database Project

Goal: To implement an AI model to efficiently gather a researcher's expertise based on their abstracts in the Astrophysics Data System (ADS). The AI model results will speed up the process of obtaining the best possible pool of proposal reviewers with relevant subject matter expertise. We will ultimatley input the names already in SMS, a database system used to create review panels, into the AI model and populate the expertise field with our results.


The code build's off of Mallory Helfenbein's (NASA HQ intern 2023) ReviewerExtractor codeV2. You must create an ADS account and obtain an api token: https://ui.adsabs.harvard.edu/user/settings/token. We input a list of researcher names into the codeV2 which searches by first author in ADS, gather their abstracts from 2003 to 2030, and returns the top 10 words, bigrams, and trigrams (give example sheet of researcher names for the github). From these n-grams, create a combined top words list.


We used the llama3-70b-8192 model in groqCloud. You must create a groqCloud account and obtain an api key: https://console.groq.com/keys. The llama model takes in the combined top words for each researcher and will determine the expertise chosen from the AAS keywords: https://journals.aas.org/keywords-2013/. We first prompted the model to determine the general topics and then asked for the associated subtopics.

Files Needed:
- ADSsearcherpkg: Python file that has all of the functions used to find the expertises of the authors and produce an organized data frame with each row being an individual author and columns: 'Input Author','Input Institution', 'First Author', 'Bibcode', 'Title', 'Publication Date', 'Keywords', 'Affiliations', 'Abstract', 'Data Type'
- TextAnalysis.py: Python file that has all the functions in order to determine the top words, bigrams and trigrams in each publication.
- stopwords.txt: Text file that has a list of the stop words for language processing.
- LlamaModel.ipynb: A notebook that contains the llama model

Limitations:
- We are searching only by first author names and if the name is common the expertise may not be accurate --> maybe for those people input institution as well (see codeV2)
- We are trusting the formatting of the llama model output but there may be cases where the format is different so manual work may be required, such as re-running certain researchers through the model again
- It takes about 4 minutes to run 10 researchers
- Rate limits in groqCloud: 30	requests per minute




## Import Packages

*   pandas 1.5.3
*   groqCloud api
*   mount to google drive
*   nltk for n-grams
*   ads api
*   TextAnalysis.py
*   stopwords.txt (to create meaningful N-grams)
*   ADSsearcherpkg.py










In [1]:
#We designed the code to work with Pandas 1.5.3
import pandas as pd
print(pd. __version__)

#If the Pandas version differs from 1.5.3, run the following:
!pip install pandas==1.5.3 --user

1.5.3


In [2]:
!pip install groq
%env GROQ_API_KEY=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 #Insert API key

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.0 MB/s eta 0:00:00
env: GROQ_API_KEY=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 #Insert API key


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#imports for n-grams
import requests
from urllib.parse import urlencode, quote_plus
import numpy as np
import sys

import nltk
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [5]:
!pip install ads

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.7 MB/s eta 0:00:00


In [4]:
path_stop= '/content/drive/MyDrive/Colab Notebooks/SMS/'
stop_file='stopwords.txt'
stop_dir=path_stop+stop_file
sys.path.append(path_stop)

In [7]:
#For the TextAnalysis File, please refer to M. Volze et al. 2023
import TextAnalysis as TA
import ADSsearcherpkg as AP

In [ ]:
#Grabbing the Data
SMS_file = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/SMS/SMS Input.csv')
third_last_1000 = SMS_file[-2500:-1500] #change range to get a subset of the data

file_name = '/content/drive/MyDrive/Colab Notebooks/SMS/3853-4853.csv'
third_last_1000.to_csv(file_name, index=False)

In [ ]:
token = 'gwAKtZG02bPL159rA6kM8clnR15iHs5OJPYyLzSZ' #Insert your API token

## Reviewer Extractor

Import a file of researcher names as a csv file (formatted Last Name, First Name) and run the names through ADS Search. This will search by first author the abstracts of a researcher from 2003 to 2030.

In [ ]:
file_name = '/content/drive/MyDrive/Colab Notebooks/SMS/3853-4853.csv'
sample_df = pd.read_csv(file_name)
sample_df

,Name,Title,LastName,Organization,Email,Expertise,Phone,MiddleName,FirstName,Notes,ID
0,"Bryan, Marta",Marta,Bryan,NaN,martalbryan@berkeley.edu,"ExoPlanet, debris disks, gas giants",NaN,NaN,NaN,NaN,3352
1,"Volonteri, Marta",Marta,Volonteri,Self,martav@iap.fr,NaN,734-647-4212,NaN,NaN,NaN,814
2,"Martens, Petrus",Petrus,Martens,"Georgia State University Research Foundation, ...",martens@astro.gsu.edu,NaN,404-413-6612,C,NaN,NaN,2380
3,"Martin, David",David,Martin,Tufts University,martin.4096@osu.edu,"circumbinary planets, dynamics",313-652-9492,Vance,NaN,NaN,3106
4,"Cordiner, Martin",Martin,Cordiner,Catholic University Of America,martin.cordiner@nasa.gov,NaN,301-286-8797,A,NaN,NaN,2646
...,...,...,...,...,...,...,...,...,...,...,...
995,"Nelson, Richard",Richard,Nelson,"Queen Mary, University of London",r.p.nelson@qmul.ac.uk,NaN,NaN,NaN,NaN,NaN,4322
996,"Walker, Reddick",Reddick,Walker,UNIVERSITY OF FLORIDA,r.walker@ufl.edu,NaN,352-294-3072,R,NaN,NaN,6621
997,"Fortenberry, Ryan",Ryan,Fortenberry,University Of Mississippi,r410@olemiss.edu,NaN,662-915-1687,C,NaN,NaN,6307
998,"Fortenberry, Ryan",Ryan,Fortenberry,NaN,r410@vt.edu,NaN,662-915-1687,C,NaN,NaN,1046


In [ ]:
#ADS Search
datf=AP.run_file_names(filename= file_name,
               token=token, stop_dir=stop_dir)

datf

I will go through each name in the list. Name should be formatted in a single column called "Last, First".  We will search by default any pubblication between 2003 and 2030 by these authors, independently of the institutions they were  affiliated to. 

Bryan, Marta
I will search for every paper who first authors is Bryan, Marta and has published between 2003 and 2030. 

I am now querying ADS.



/content/drive/MyDrive/Colab Notebooks/SMS/ADSsearcherpkg.py:148: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df= final_df.append(data1, ignore_index= True)


1 iterations done
Volonteri, Marta
I will search for every paper who first authors is Volonteri, Marta and has published between 2003 and 2030. 

I am now querying ADS.



/content/drive/MyDrive/Colab Notebooks/SMS/ADSsearcherpkg.py:148: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df= final_df.append(data1, ignore_index= True)


2 iterations done
Martens, Petrus
I will search for every paper who first authors is Martens, Petrus and has published between 2003 and 2030. 

I am now querying ADS.



...

/content/drive/MyDrive/Colab Notebooks/SMS/ADSsearcherpkg.py:148: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df= final_df.append(data1, ignore_index= True)


999 iterations done
Beaton, Rachael
I will search for every paper who first authors is Beaton, Rachael and has published between 2003 and 2030. 

I am now querying ADS.



/content/drive/MyDrive/Colab Notebooks/SMS/ADSsearcherpkg.py:148: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df= final_df.append(data1, ignore_index= True)


1000 iterations done


,Input Author,Input Institution,First Author,Bibcode,Title,Publication Date,Keywords,Affiliations,Abstract,Identifier,Top 10 Words,Top 10 Bigrams,Top 10 Trigrams,Data Type
0,"Bryan, Marta","None, None, None, None, None, None, None, None...","Bryan, Marta L., Bryan, Marta, Bryan, Marta L....","2024ApJ...968L..25B, 2024AAS...24333201B, 2021...",Friends Not Foes: Strong Correlation between I...,"2024-06-00, 2024-02-00, 2021-11-00, 2020-12-00...","Exoplanet astronomy, Exoplanet evolution, Exop...",David A. Dunlap Department of Astronomy &amp; ...,The connection between outer gas giants and in...,"10.3847/2041-8213/ad5013, 2024arXiv240308873B,...","[(companion, 81), (system, 80), (planet, 79), ...","[((gas, giant), 40), ((rotation, rate), 20), (...","[((gas, giant, planets), 13), ((planetary, mas...","Dirty, Clean, Dirty, Dirty, Dirty, Dirty, Dirt..."
1,"Volonteri, Marta","None, None, None, None, None, None, None, None...","Volonteri, Marta, Volonteri, Marta, Volonteri,...","2023MNRAS.521..241V, 2022MNRAS.514..640V, 2022...",What if young z &gt; 9 JWST galaxies hosted ma...,"2023-05-00, 2022-07-00, 2022-05-00, 2021-12-00...","galaxies: active, galaxies: evolution, galaxie...","Institut d'Astrophysique de Paris, Sorbonne Un...",JWST is discovering star-forming 'candidate' g...,"10.1093/mnras/stad499, 2022arXiv221204710V, ar...","[(hole, 240), (black, 221), (galaxy, 195), (ma...","[((black, hole), 221), ((massive, black), 72),...","[((massive, black, holes), 45), ((massive, bla...","Clean, Clean, Dirty, Dirty, Dirty, Clean, Dirt..."
2,"Martens, Petrus","None, None, None, None, None, None, None, None...","Martens, Petrus C., Martens, Petrus, Martens, ...","2023AGUFMSH33B..03M, 2023SPD....5410711M, 2022...","Topological Solitons, or ""Floaters"", Model the...","2023-12-00, 2023-10-00, 2022-12-00, 2019-12-00...",", , , 7536 Solar activity cycle, SOLAR PHYSICS...","Georgia State University, Atlanta, United Stat...",Switchbacks are a major discovery of the Parke...,"2023AGUFMSH33B..03M, 2023SPD....5410711M, 2022...","[(solar, 94), (data, 70), (feature, 46), (modu...","[((magnetic, field), 20), ((active, region), 1...","[((automated, feature, recognition), 10), ((fe...","Dirty, Dirty, Dirty, Dirty, Dirty, Dirty, Dirt..."
3,"Martin, David","None, None, None, None, None, None, None, None...","Martin, David, Martin, David V., Martin, D. Ch...","2024ESS.....561407M, 2024MNRAS.528..963M, 2023...",BEBOP - The first circumbinary planets with RV...,"2024-04-00, 2024-02-00, 2023-11-00, 2023-03-00...",", techniques: photometric, techniques: radial ...","Tufts University, Department of Physics &amp; ...",The best way to test planet formation is to fi...,"2024ESS.....561407M, 2024MNRAS.528..963M, 10.4...","[(planet, 92), (binary, 63), (star, 57), (mass...","[((circumbinary, planet), 35), ((eclipsing, bi...","[((star, formation, rate), 7), ((transiting, c...","Dirty, Dirty, Dirty, Dirty, Dirty, Dirty, Dirt..."
4,"Cordiner, Martin","None, None, None, None, None, None, None, None...","Cordiner, M. A., Cordiner, Martin A., Cordiner...","2024arXiv240405525C, 2024arXiv240302258C, 2024...",ALMA Spectroscopy of Europa: A Search for Acti...,"2024-04-00, 2024-03-00, 2024-02-00, 2023-12-00...",Astrophysics - Earth and Planetary Astrophysic...,"-, -, Catholic University of America, Solar Sy...",The subsurface ocean of Europa is a high prior...,"arXiv:2404.05525, 10.48550/arXiv.2404.05525, 2...","[(interstellar, 107), (observation, 101), (cn,...","[((diffuse, interstellar), 33), ((lambda, lamb...","[((atacama, large, millimeter), 27), ((large, ...","Dirty, Dirty, Dirty, Dirty, Dirty, Dirty, Dirt..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
924,"Nelson, Richard","None, None, None, None, None, None, None, None...","Nelson, R. H., Nelson, R. H., Nelson, R., Nels...","2023NewAR..9701684N, 2023RMxAA..59..201N, 2023...",A critical review of period analyses and impli...,"2023-12-00, 2023-10-00, 2023-03-00, 2023-02-00...",", binaries: eclipsing, binarie

In [ ]:
#No ADS search results for these people; saves to separate excel file
ads_no_results = sample_df[sample_df["Name"].isin(datf["Input Author"]) == False]
ads_no_results.to_excel(path_stop+'ADS_no_results12.xlsx', index=False)

In [ ]:
import itertools
def topwords(top10words, top10bigrams, top10trigrams):
    '''
    Takes in a list of top 10 words, bigrams, and trigrams and returns a combined list
    '''
    # Handle the case where input might not be lists of tuples
    if isinstance(top10words, str):
        top10words = eval(top10words) # Safely evaluate string representation of list
    if isinstance(top10bigrams, str):
        top10bigrams = eval(top10bigrams)
    if isinstance(top10trigrams, str):
        top10trigrams = eval(top10trigrams)

    topwords = [word for word, _ in top10words]
    topbigrams = [' '.join(words) for words, _ in top10bigrams]
    toptrigrams = [' '.join(words) for words, _ in top10trigrams]

    lst = [topwords, topbigrams, toptrigrams]
    single_list = list(itertools.chain(*lst))
    return single_list

Combine the N-grams into a single list using the function above. Also append the author's institution and the number of research papers they wrote as first author from 2003 to 2030.

In [ ]:
datf["Top Combined Words"] = datf.apply(lambda x: topwords(x["Top 10 Words"], x["Top 10 Bigrams"], x['Top 10 Trigrams']), axis=1)
#datf["Organization"] = sample_df["Institution"]
datf["Number of Papers"] = datf["Bibcode"].str.split(",").str.len()
datf

,Input Author,Input Institution,First Author,Bibcode,Title,Publication Date,Keywords,Affiliations,Abstract,Identifier,Top 10 Words,Top 10 Bigrams,Top 10 Trigrams,Data Type,Top Combined Words,Number of Papers
0,"Bryan, Marta","None, None, None, None, None, None, None, None...","Bryan, Marta L., Bryan, Marta, Bryan, Marta L....","2024ApJ...968L..25B, 2024AAS...24333201B, 2021...",Friends Not Foes: Strong Correlation between I...,"2024-06-00, 2024-02-00, 2021-11-00, 2020-12-00...","Exoplanet astronomy, Exoplanet evolution, Exop...",David A. Dunlap Department of Astronomy &amp; ...,The connection between outer gas giants and in...,"10.3847/2041-8213/ad5013, 2024arXiv240308873B,...","[(companion, 81), (system, 80), (planet, 79), ...","[((gas, giant), 40), ((rotation, rate), 20), (...","[((gas, giant, planets), 13), ((planetary, mas...","Dirty, Clean, Dirty, Dirty, Dirty, Dirty, Dirt...","[companion, system, planet, mass, giant, gas, ...",25
1,"Volonteri, Marta","None, None, None, None, None, None, None, None...","Volonteri, Marta, Volonteri, Marta, Volonteri,...","2023MNRAS.521..241V, 2022MNRAS.514..640V, 2022...",What if young z &gt; 9 JWST galaxies hosted ma...,"2023-05-00, 2022-07-00, 2022-05-00, 2021-12-00...","galaxies: active, galaxies: evolution, galaxie...","Institut d'Astrophysique de Paris, Sorbonne Un...",JWST is discovering star-forming 'candidate' g...,"10.1093/mnras/stad499, 2022arXiv221204710V, ar...","[(hole, 240), (black, 221), (galaxy, 195), (ma...","[((black, hole), 221), ((massive, black), 72),...","[((massive, black, holes), 45), ((massive, bla...","Clean, Clean, Dirty, Dirty, Dirty, Clean, Dirt...","[hole, black, galaxy, mass, merger, massive, m...",73
2,"Martens, Petrus","None, None, None, None, None, None, None, None...","Martens, Petrus C., Martens, Petrus, Martens, ...","2023AGUFMSH33B..03M, 2023SPD....5410711M, 2022...","Topological Solitons, or ""Floaters"", Model the...","2023-12-00, 2023-10-00, 2022-12-00, 2019-12-00...",", , , 7536 Solar activity cycle, SOLAR PHYSICS...","Georgia State University, Atlanta, United Stat...",Switchbacks are a major discovery of the Parke...,"2023AGUFMSH33B..03M, 2023SPD....5410711M, 2022...","[(solar, 94), (data, 70), (feature, 46), (modu...","[((magnetic, field), 20), ((active, region), 1...","[((automated, feature, recognition), 10), ((fe...","Dirty, Dirty, Dirty, Dirty, Dirty, Dirty, Dirt...","[solar, data, feature, module, magnetic, sdo, ...",38
3,"Martin, David","None, None, None, None, None, None, None, None...","Martin, David, Martin, David V., Martin, D. Ch...","2024ESS.....561407M, 2024MNRAS.528..963M, 2023...",BEBOP - The first circumbinary planets with RV...,"2024-04-00, 2024-02-00, 2023-11-00, 2023-03-00...",", techniques: photometric, techniques: radial ...","Tufts University, Department of Physics &amp; ...",The best way to test planet formation is to fi...,"2024ESS.....561407M, 2024MNRAS.528..963M, 10.4...","[(planet, 92), (binary, 63), (star, 57), (mass...","[((circumbinary, planet), 35), ((eclipsing, bi...","[((star, formation, rate), 7), ((transiting, c...","Dirty, Dirty, Dirty, Dirty, Dirty, Dirty, Dirt...","[planet, binary, star, mass, circumbinary, gal...",66
4,"Cordiner, Martin","None, None, None, None, None, None, None, None...","Cordiner, M. A., Cordiner, Martin A., Cordiner...","2024arXiv240405525C, 2024arXiv240302258C, 2024...",ALMA Spectroscopy of Europa: A Search for Acti...,"2024-04-00, 2024-03-00, 2024-02-00, 2023-12-00...",Astrophysics - Earth and Planetary Astrophysic...,"-, -, Catholic University of America, Solar Sy...",The subsurface ocean of Europa is a high prior...,"arXiv:2404.05525, 10.48550/arXiv.2404.05525, 2...","[(interstellar, 107), (observation, 101), (cn,...","[((diffuse, interstellar), 33), ((lambda, lamb...","[((atacama, large, millimeter), 27), ((large, ...","Dirty, Dirty, Dirty, Dirty, Dirty, Dirty, Dirt...","[interstellar, observation, cn, coma, molecule...",95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [ ]:
#drop duplicates
datf = datf.astype(str) # convert to string
datf.drop_duplicates(inplace=True)
datf

,Input Author,Input Institution,First Author,Bibcode,Title,Publication Date,Keywords,Affiliations,Abstract,Identifier,Top 10 Words,Top 10 Bigrams,Top 10 Trigrams,Data Type,Top Combined Words,Number of Papers
0,"Bryan, Marta","None, None, None, None, None, None, None, None...","Bryan, Marta L., Bryan, Marta, Bryan, Marta L....","2024ApJ...968L..25B, 2024AAS...24333201B, 2021...",Friends Not Foes: Strong Correlation between I...,"2024-06-00, 2024-02-00, 2021-11-00, 2020-12-00...","Exoplanet astronomy, Exoplanet evolution, Exop...",David A. Dunlap Department of Astronomy &amp; ...,The connection between outer gas giants and in...,"10.3847/2041-8213/ad5013, 2024arXiv240308873B,...","[('companion', 81), ('system', 80), ('planet',...","[(('gas', 'giant'), 40), (('rotation', 'rate')...","[(('gas', 'giant', 'planets'), 13), (('planeta...","Dirty, Clean, Dirty, Dirty, Dirty, Dirty, Dirt...","['companion', 'system', 'planet', 'mass', 'gia...",25
1,"Volonteri, Marta","None, None, None, None, None, None, None, None...","Volonteri, Marta, Volonteri, Marta, Volonteri,...","2023MNRAS.521..241V, 2022MNRAS.514..640V, 2022...",What if young z &gt; 9 JWST galaxies hosted ma...,"2023-05-00, 2022-07-00, 2022-05-00, 2021-12-00...","galaxies: active, galaxies: evolution, galaxie...","Institut d'Astrophysique de Paris, Sorbonne Un...",JWST is discovering star-forming 'candidate' g...,"10.1093/mnras/stad499, 2022arXiv221204710V, ar...","[('hole', 240), ('black', 221), ('galaxy', 195...","[(('black', 'hole'), 221), (('massive', 'black...","[(('massive', 'black', 'holes'), 45), (('massi...","Clean, Clean, Dirty, Dirty, Dirty, Clean, Dirt...","['hole', 'black', 'galaxy', 'mass', 'merger', ...",73
2,"Martens, Petrus","None, None, None, None, None, None, None, None...","Martens, Petrus C., Martens, Petrus, Martens, ...","2023AGUFMSH33B..03M, 2023SPD....5410711M, 2022...","Topological Solitons, or ""Floaters"", Model the...","2023-12-00, 2023-10-00, 2022-12-00, 2019-12-00...",", , , 7536 Solar activity cycle, SOLAR PHYSICS...","Georgia State University, Atlanta, United Stat...",Switchbacks are a major discovery of the Parke...,"2023AGUFMSH33B..03M, 2023SPD....5410711M, 2022...","[('solar', 94), ('data', 70), ('feature', 46),...","[(('magnetic', 'field'), 20), (('active', 'reg...","[(('automated', 'feature', 'recognition'), 10)...","Dirty, Dirty, Dirty, Dirty, Dirty, Dirty, Dirt...","['solar', 'data', 'feature', 'module', 'magnet...",38
3,"Martin, David","None, None, None, None, None, None, None, None...","Martin, David, Martin, David V., Martin, D. Ch...","2024ESS.....561407M, 2024MNRAS.528..963M, 2023...",BEBOP - The first circumbinary planets with RV...,"2024-04-00, 2024-02-00, 2023-11-00, 2023-03-00...",", techniques: photometric, techniques: radial ...","Tufts University, Department of Physics &amp; ...",The best way to test planet formation is to fi...,"2024ESS.....561407M, 2024MNRAS.528..963M, 10.4...","[('planet', 92), ('binary', 63), ('star', 57),...","[(('circumbinary', 'planet'), 35), (('eclipsin...","[(('star', 'formation', 'rate'), 7), (('transi...","Dirty, Dirty, Dirty, Dirty, Dirty, Dirty, Dirt...","['planet', 'binary', 'star', 'mass', 'circumbi...",66
4,"Cordiner, Martin","None, None, None, None, None, None, None, None...","Cordiner, M. A., Cordiner, Martin A., Cordiner...","2024arXiv240405525C, 2024arXiv240302258C, 2024...",ALMA Spectroscopy of Europa: A Search for Acti...,"2024-04-00, 2024-03-00, 2024-02-00, 2023-12-00...",Astrophysics - Earth and Planetary Astrophysic...,"-, -, Catholic University of America, Solar Sy...",The subsurface ocean of Europa is a high prior...,"arXiv:2404.05525, 10.48550/arXiv.2404.05525, 2...","[('interstellar', 107), ('observation', 101), ...","[(('diffuse', 'interstellar'), 33), (('lambda'...","[(('atacama', 'large', 'millimeter'), 27), (('...","Dirty, Dirty, Dirty, Dirty, Dirty, Dirty, Dirt...","['interstellar', 'observation', 'cn', 'coma', ...",95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [ ]:
#Save ADS search results file
datf.to_excel(path_stop+'ADS_results12.xlsx', index=False)

In [10]:
#import pandas as pd
# @title Re-Import ADS Search Results (if code errors)
#Can re-import ADS Search results file here if the code errors
df = pd.read_excel(path_stop+'ADS_results12.xlsx')
datf = df[728:]
datf

,Input Author,Input Institution,First Author,Bibcode,Title,Publication Date,Keywords,Affiliations,Abstract,Identifier,Top 10 Words,Top 10 Bigrams,Top 10 Trigrams,Data Type,Top Combined Words,Number of Papers
728,"Korngut, Phil","None, None, None","Korngut, P. M., Korngut, P. M., Korngut, P. M.","2022ApJ...926..133K, 2013ApJS..207...34K, 2011...",Inferred Measurements of the Zodiacal Light Ab...,"2022-02-00, 2013-08-00, 2011-06-00","343, Astrophysics - Earth and Planetary Astrop...","Department of Physics, California Institute of...",Scattered sunlight from the interplanetary dus...,"2021arXiv210407104K, 10.3847/1538-4357/ac44ff,...","[('measurement', 8), ('model', 7), ('zl', 6), ...","[(('interplanetary', 'dust'), 2), (('solar', '...","[(('extragalactic', 'background', 'light'), 2)...","Dirty, Dirty, Dirty","['measurement', 'model', 'zl', 'data', 'sze', ...",3
729,"Plambeck, Richard","None, None, None, None, None, None, None","Plambeck, R. L., Plambeck, R. L., Plambeck, R....","2016ApJ...833..219P, 2014ApJ...797...66P, 2013...",ALMA Observations of Orion Source I at 350 and...,"2016-12-00, 2014-12-00, 2013-03-00, 2009-10-00...","ISM: individual objects: Orion-KL, radio conti...","Radio Astronomy Laboratory, University of Cali...",Orion Source I (“SrcI”) is the protostar at th...,"2016arXiv160800996P, arXiv:1608.00996, 10.4855...","[('srci', 11), ('line', 11), ('ghz', 9), ('dis...","[(('position', 'angle'), 7), (('recombination'...","[(('hypercompact', 'ii', 'region'), 2), (('bec...","Dirty, Dirty, Dirty, Dirty, Dirty, Dirty, Dirty","['srci', 'line', 'ghz', 'disk', 'source', 'emi...",7
730,"La Plante, Paul","None, None, None, None, None, None, None, None...","La Plante, Paul, La Plante, Paul, La Plante, P...","2023ApJ...944...59L, 2022AAS...24031408L, 2022...",Prospects for 21 cm Galaxy Cross-correlations ...,"2023-02-00, 2022-06-00, 2022-04-00, 2021-07-00...","Cosmology, Large-scale structure of the univer...","Department of Astronomy, University of Califor...",The cross-correlation between the 21 cm field ...,"2022arXiv220509770L, arXiv:2205.09770, 10.3847...","[('reionization', 62), ('hera', 26), ('cm', 24...","[(('epoch', 'reionization'), 18), (('power', '...","[(('hydrogen', 'epoch', 'reionization'), 8), (...","Clean, Clean, Clean, Dirty, Dirty, Clean, Dirt...","['reionization', 'hera', 'cm', 'data', 'epoch'...",17
731,"Plavchan, Peter","None, None, None, None, None, None, None, None...","Plavchan, Peter, Plavchan, Peter, Plavchan, Pe...","2024arXiv240102039P, 2023AAS...24233004P, 2020...",Analytic relations assessing the impact of pre...,"2024-01-00, 2023-06-00, 2020-07-00, 2020-06-00...",Astrophysics - Earth and Planetary Astrophysic...,"-, George Mason University, Department of Phys...",The Habitable Worlds Observatory will attempt ...,"2024arXiv240102039P, 10.48550/arXiv.2401.02039...","[('infrared', 83), ('star', 68), ('dwarf', 67)...","[(('radial', 'velocity'), 51), (('debris', 'di...","[(('nasa', 'infrared', 'telescope'), 14), (('i...","Clean, Clean, Dirty, Clean, Dirty, Dirty, Clea...","['infrared', 'star', 'dwarf', 'planet', 'mass'...",55
732,"Lightsey, Paul","None, None, None, None, None, None, None, None...","Lightsey, Paul A., Lightsey, Paul A., Lightsey...","2018SPIE10698E..13L, 2018SPIE10698E..45L, 2018...","Optical budgeting for LUVOIR, Stray light over...","2018-07-00, 2018-07-00, 2018-07-00, 2018-00-00...",", , , , , , , , , , , , , , , , , , ,",Ball Aerospace &amp; Technologies Corp. (Unite...,Future large astronomical telescopes in space ...,"10.1117/12.2312256, 2018SPIE10698E..13L, 2018S...","[('telescope', 64), ('space', 41), ('mirror', ...","[(('stray', 'light'), 26), (('space', 'telesco...","[(('james', 'webb', 'space'), 14), (('webb', '...","Dirty, Dirty, Dirty, Dirty, Dirty, Dirty, Dirt...","['telescope', 'space', 'mirror', 'light', 'sky...",20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
797,"Martinez, Raquel","None, None, None, None, None, None, None, None...","Martinez, Ra

## Preparing the Llama Model

*   Create a dictionary of topics and their subtopics
*   Create the topic and subtopic prompts for the model
*   extract_topics function to extract the expertise from the model's response
*   generate_subtopics function to generate the subtopics with the model






In [11]:
# @title Subtopics Dictionary
d = {
"physical data and processes":
"""
acceleration of particles
accretion, accretion disks
asteroseismology
astrobiology
astrochemistry
astroparticle physics
atomic data
atomic processes
black hole physics
chaos
conduction
convection
dense matter
diffusion
dynamo
elementary particles
equation of state
gravitation
gravitational lensing: strong
gravitational lensing: weak
gravitational lensing: micro
gravitational waves
hydrodynamics
instabilities
line: formation
line: identification
line: profiles
magnetic fields
magnetic reconnection
magnetohydrodynamics (MHD)
masers
molecular data
molecular processes
neutrinos
nuclear reactions, nucleosynthesis, abundances
opacity
plasmas
polarization
radiation: dynamics
radiation mechanisms: general
radiation mechanisms: non-thermal
radiation mechanisms: thermal
radiative transfer
relativistic processes
scattering
shock waves
solid state: refractory
solid state: volatile
turbulence
waves
""",
"astronomical instrumentation methods and techniques":
"""
atmospheric effects
balloons
instrumentation: adaptive optics
instrumentation: detectors
instrumentation: high angular resolution
instrumentation: interferometers
instrumentation: miscellaneous
instrumentation: photometers
instrumentation: polarimeters
instrumentation: spectrographs
light pollution
methods: analytical
methods: data analysis
methods: laboratory: atomic
methods: laboratory: molecular
methods: laboratory: solid state
methods: miscellaneous
methods: numerical
methods: observational
methods: statistical
site testing
space vehicles
space vehicles: instruments
techniques: high angular resolution
techniques: image processing
techniques: imaging spectroscopy
techniques: interferometric
techniques: miscellaneous
techniques: photometric
techniques: polarimetric
techniques: radar astronomy
techniques: radial velocities
techniques: spectroscopic
telescopes
""",
"astronomical databases":
"""
astronomical databases: miscellaneous
atlases
catalogs
surveys
virtual observatory tools
""",
"astrometry and celestial mechanics":
"""
astrometry
celestial mechanics
eclipses
ephemerides
occultations
parallaxes
proper motions
reference systems
time
""",
"the sun":
"""
Sun: abundances
Sun: activity
Sun: atmosphere
Sun: chromosphere
Sun: corona
Sun: coronal mass ejections (CMEs)
Sun: evolution
Sun: faculae, plages
Sun: filaments, prominences
Sun: flares
Sun: fundamental parameters
Sun: general
Sun: granulation
Sun: helioseismology
Sun: heliosphere
Sun: infrared
Sun: interior
Sun: magnetic fields
Sun: oscillations
Sun: particle emission
Sun: photosphere
Sun: radio radiation
Sun: rotation
(Sun:) solar–terrestrial relations
(Sun:) solar wind
(Sun:) sunspots
Sun: transition region
Sun: UV radiation
Sun: X-rays, gamma rays
""",
"planetary systems":
"""
comets: general
comets: individual (…, …)
Earth
interplanetary medium
Kuiper belt: general
Kuiper belt objects: individual (…, …)
meteorites, meteors, meteoroids
minor planets, asteroids: general
minor planets, asteroids: individual (…, …)
Moon
Oort Cloud
planets and satellites: atmospheres
planets and satellites: aurorae
planets and satellites: composition
planets and satellites: detection
planets and satellites: dynamical evolution and stability
planets and satellites: formation
planets and satellites: fundamental parameters
planets and satellites: gaseous planets
planets and satellites: general
planets and satellites: individual (…, …)
planets and satellites: interiors
planets and satellites: magnetic fields
planets and satellites: oceans
planets and satellites: physical evolution
planets and satellites: rings
planets and satellites: surfaces
planets and satellites: tectonics
planets and satellites: terrestrial planets
protoplanetary disks
planet–disk interactions
planet–star interactions
zodiacal dust
""",
"stars":
"""
stars: abundances
stars: activity
stars: AGB and post-AGB
stars: atmospheres
(stars:) binaries (including multiple): close
(stars:) binaries: eclipsing
(stars:) binaries: general
(stars:) binaries: spectroscopic
(stars:) binaries: symbiotic
(stars:) binaries: visual
stars: black holes
(stars:) blue stragglers
(stars:) brown dwarfs
stars: carbon
stars: chemically peculiar
stars: chromospheres
(stars:) circumstellar matter
stars: coronae
stars: distances
stars: dwarf novae
stars: early-type
stars: emission-line, Be
stars: evolution
stars: flare
stars: formation
stars: fundamental parameters
stars: general
(stars:) gamma-ray burst: general
(stars:) gamma-ray burst: individual (…, …)
(stars:) Hertzsprung–Russell and C–M diagrams
stars: horizontal-branch
stars: imaging
stars: individual (…, …)
stars: interiors
stars: jets
stars: kinematics and dynamics
stars: late-type
stars: low-mass
stars: luminosity function, mass function
stars: magnetars
stars: magnetic field
stars: massive
stars: mass-loss
stars: neutron
(stars:) novae, cataclysmic variables
stars: oscillations (including pulsations)
stars: peculiar (except chemically peculiar)
(stars:) planetary systems
stars: Population II
stars: Population III
stars: pre-main sequence
stars: protostars
(stars:) pulsars: general
(stars:) pulsars: individual (…, …)
stars: rotation
stars: solar-type
(stars:) starspots
stars: statistics
(stars:) subdwarfs
(stars:) supergiants
(stars:) supernovae: general
(stars:) supernovae: individual (…, …)
stars: variables: Cepheids
stars: variables: delta Scuti
stars: variables: general
stars: variables: RR Lyrae
stars: variables: S Doradus
stars: variables: T Tauri, Herbig Ae/Be
(stars:) white dwarfs
stars: winds, outflows
stars: Wolf–Rayet
""",
"interstellar medium (ism) nebulae":
"""
ISM: abundances
ISM: atoms
ISM: bubbles
ISM: clouds
(ISM:) cosmic rays
(ISM:) dust, extinction
(ISM:) evolution
ISM: general
(ISM:) HII regions
(ISM:) Herbig–Haro objects
ISM: individual objects (…, …) (except
planetary nebulae)
ISM: jets and outflows
ISM: kinematics and dynamics
ISM: lines and bands
ISM: magnetic fields
ISM: molecules
(ISM:) planetary nebulae: general
(ISM:) planetary nebulae: individual (…, …)
(ISM:) photon-dominated region (PDR)
ISM: structure
ISM: supernova remnants
""",
"the galaxy":
"""
Galaxy: abundances
Galaxy: bulge
Galaxy: center
Galaxy: disk
Galaxy: evolution
Galaxy: formation
Galaxy: fundamental parameters
Galaxy: general
(Galaxy:) globular clusters: general
(Galaxy:) globular clusters: individual (…, …)
Galaxy: halo
(Galaxy:) local interstellar matter
Galaxy: kinematics and dynamics
Galaxy: nucleus
(Galaxy:) open clusters and associations: general
(Galaxy:) open clusters and associations: individual (…, …)
(Galaxy:) solar neighborhood
Galaxy: stellar content
Galaxy: structure
""",
"galaxies":
"""
galaxies: abundances
galaxies: active
(galaxies:) BL Lacertae objects: general
(galaxies:) BL Lacertae objects: individual (…, …)
galaxies: bulges
galaxies: clusters: general
galaxies: clusters: individual (…, …)
galaxies: clusters: intracluster medium
galaxies: distances and redshifts
galaxies: dwarf
galaxies: elliptical and lenticular, cD
galaxies: evolution
galaxies: formation
galaxies: fundamental parameters
galaxies: general
galaxies: groups: general
galaxies: groups: individual (…, …)
galaxies: halos
galaxies: high-redshift
galaxies: individual (…, …)
galaxies: interactions
(galaxies:) intergalactic medium
galaxies: irregular
galaxies: ISM
galaxies: jets
galaxies: kinematics and dynamics
(galaxies:) Local Group
galaxies: luminosity function, mass function
(galaxies:) Magellanic Clouds
galaxies: magnetic fields
galaxies: nuclei
galaxies: peculiar
galaxies: photometry
(galaxies:) quasars: absorption lines
(galaxies:) quasars: emission lines
(galaxies:) quasars: general
(galaxies:) quasars: individual (…, …)
(galaxies:) quasars: supermassive black holes
galaxies: Seyfert
galaxies: spiral
galaxies: starburst
galaxies: star clusters: general
galaxies: star clusters: individual (…, …)
galaxies: star formation
galaxies: statistics
galaxies: stellar content
galaxies: structure
""",
"cosmology":
"""
(cosmology:) cosmic background radiation
(cosmology:) cosmological parameters
cosmology: miscellaneous
cosmology: observations
cosmology: theory
(cosmology:) dark ages, reionization, first stars
(cosmology:) dark matter
(cosmology:) dark energy
(cosmology:) diffuse radiation
(cosmology:) distance scale
(cosmology:) early universe
(cosmology:) inflation
(cosmology:) large-scale structure of universe
(cosmology:) primordial nucleosynthesis
""",
"resolved and unresolved sources as a function of wavelength":
"""
gamma rays: diffuse background
gamma rays: galaxies
gamma rays: galaxies: clusters
gamma rays: general
gamma rays: ISM
gamma rays: stars
infrared: diffuse background
infrared: galaxies
infrared: general
infrared: ISM
infrared: planetary systems
infrared: stars
radio continuum: galaxies
radio continuum: general
radio continuum: ISM
radio continuum: planetary systems
radio continuum: stars
radio lines: galaxies
radio lines: general
radio lines: ISM
radio lines: planetary systems
radio lines: stars
submillimeter: diffuse background
submillimeter: galaxies
submillimeter: general
submillimeter: ISM
submillimeter: planetary systems
submillimeter: stars
ultraviolet: galaxies
ultraviolet: general
ultraviolet: ISM
ultraviolet: planetary systems
ultraviolet: stars
X-rays: binaries
X-rays: bursts
X-rays: diffuse background
X-rays: galaxies
X-rays: galaxies: clusters
X-rays: general
X-rays: individual (…, …)
X-rays: ISM
X-rays: stars
"""
}

In [12]:
prompt_topic = """
You are a scientist determining the areas of expertise of a person based on a list of top words from their abstracts pulled from the NASA Astrophysics Data System (ADS). You will be provided a list of top words to determine the topic or topics the person is an expert in. The topics are listed below.

Topics list:
physical data and processes
astronomical instrumentation methods and techniques
astronomical databases
astrometry and celestial mechanics
the sun
planetary systems
stars
interstellar medium (ism) nebulae
the galaxy
galaxies
cosmology
resolved and unresolved sources as a function of wavelength

Now please determine accurately the topic or topics based on these top words and provide evidence. You MUST ONLY choose from the topics list above. Please list out the topics in a python list format first. Here are some examples of the expected format, [galaxies, the sun, cosmology], [astronomical databases], [interstellar medium (ism) nebulae, astronomical instrumentation methods and techniques, resolved and unresolved sources as a function of wavelength, stars]. Note these examples contain topics chosen ONLY from the topics list and do not include the ` character. Then provide evidence by explaining which of the top words correspond with the topic. Here are the top words.:
"""

def prompt_subtopic(subtopic):
  prompt = """
  You are a scientist determining the specific areas of expertise of a person based on a list of top words from their abstracts pulled from the NASA Astrophysics Data System (ADS) and their associated general topic. You will be provided a list of top words and the general topic to determine the subtopic or subtopics the person is an expert in. The subtopics are listed below.

  Subopics list:
  """ + subtopic + """
  Now please determine accurately the subtopic or subtopics based on these top words and general topic. You MUST ONLY choose from the subtopics list above. Choose to the best of your ability and do not leave the response as none. Please list out the topic and subtopics in this format: [topic - subtopic1|subtopic2|etc]. Some examples of the expected format are [physical data and processes - astrobiology|diffusion|gravitational lensing: strong], [astrometry and celestial mechanics - occultations], [stars - (stars:) circumstellar matter|stars: flare|stars: interiors|(stars:) supernovae: general]. Note these examples contain subtopics chosen ONLY from the subtopics list for each topic.
  Then provide evidence by explaining which of the top words correspond with the subtopics. Here are the top words and the general topic.
  """
  return prompt

In [13]:
###IMPORTANT FUNCTIONS###

def extract_topics(text, topic):
  # Extract the list using a regular expression
  match = re.search(r'\[(.*?)\]', text)
  if match:
    list_string = match.group(1)
  else:
    return 'None'
  if topic == "t":
      result_list = [item.strip().lower().replace('`', '') for item in list_string.split(',')]
      return result_list
  else:
    return list_string


def generate_subtopics(topwords, topic):
    try:
      subtopics = d[topic]
    except:
      return "["+topic + " - not listed]"

    client = Groq()
    completion = client.chat.completions.create(
      model="llama3-70b-8192",
      messages=[
          {
            "role": "user",
            "content": prompt_subtopic(subtopics) + "Top Words: " + str(topwords) + " Topic: " + topic
          }
        ],
      temperature=0,
      max_tokens=3000,
      top_p=1,
      stream=True,
      stop=None,
    )
    response = "".join(chunk.choices[0].delta.content or "" for chunk in completion)
    return response

## Run the Llama Model

First open a new csv file and name it. **Note each time you run the cell below it will erase the entire sheet and have only the headers.** Then run the llama model and the csv file will update after each iteration. The model reads the topwords for a given researcher and returns the topics that match those words. Then it will run those topics and topwords through the model again but this time to gather the subtopics (gather_subtopics function is called).

In [ ]:
import csv

output_file = path_stop+"output_3853-4853.csv" #name the output file
with open(output_file, 'w', newline='', encoding='utf-8') as dynamic_csv_file:
  csv_writer = csv.writer(dynamic_csv_file)
  csv_writer.writerow(["Input Author", "Affiliations", "Combined Top Words", "Topics with Explanation", "Subtopics with Explanation", "Topics", "Subtopics", "Number of Papers"])

In [15]:
from groq import Groq
import re
import csv

client = Groq()
%env GROQ_API_KEY=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
output_file = path_stop+"output_3853-4853.csv"
num_iter = 728

for topwords in datf["Top Combined Words"]:
  completion = client.chat.completions.create(
      model="llama3-70b-8192",
      messages=[
          {
              "role": "user",
              "content": prompt_topic + str(topwords)
          }
      ],
      temperature=0,
      max_tokens=3000,
      top_p=1,
      stream=True,
      stop=None,
  )

  #Topics with explanation
  response = "".join(chunk.choices[0].delta.content or "" for chunk in completion)
  #Extract only the topic
  topics = extract_topics(response, "t")
  #Subtopics with explanation
  full_st = []
  #Extract only the subtopics
  st = []

  for topic in topics:
    subtopics = generate_subtopics(topwords, topic)
    full_st.append(subtopics)
    st.append(extract_topics(subtopics, "s"))

  #Get input author, institution, affiliations, and number of papers
  author = datf["Input Author"][num_iter]
  #inst = datf["Institution"][num_iter]
  aff = datf["Affiliations"][num_iter]
  papers = datf["Number of Papers"][num_iter]

  #Write to csv
  with open(output_file, 'a', newline='', encoding='utf-8') as dynamic_csv_file:
    csv_writer = csv.writer(dynamic_csv_file)
    csv_writer.writerow([author, aff, topwords, response, full_st, topics, st, papers])
    num_iter += 1
    print("Completed Row " + str(num_iter))

env: GROQ_API_KEY=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
Completed Row 729
Completed Row 730
Completed Row 731
Completed Row 732
Completed Row 733
Completed Row 734
Completed Row 735
Completed Row 736
Completed Row 737
Completed Row 738
Completed Row 739
Completed Row 740
Completed Row 741
Completed Row 742
Completed Row 743
Completed Row 744
Completed Row 745
Completed Row 746
Completed Row 747
Completed Row 748
Completed Row 749
Completed Row 750
Completed Row 751
Completed Row 752
Completed Row 753
Completed Row 754
Completed Row 755
Completed Row 756
Completed Row 757
Completed Row 758
Completed Row 759
Completed Row 760
Completed Row 761
Completed Row 762
Completed Row 763
Completed Row 764
Completed Row 765
Completed Row 766
Completed Row 767
Completed Row 768
Completed Row 769
Completed Row 770
Completed Row 771
Completed Row 772
Completed Row 773
Completed Row 774
Completed Row 775
Completed Row 776
Completed Row 777
Completed Row 778
Completed Row 779
Complet

## Formatting the Data
Read in the final csv file with the model results. Then format the results in the following cell and export as an excel file.

In [5]:
output_file = path_stop+"output_3853-4853.csv"
output_df = pd.read_csv(output_file)
output_df

,Input Author,Affiliations,Combined Top Words,Topics with Explanation,Subtopics with Explanation,Topics,Subtopics,Number of Papers
0,"Bryan, Marta",David A. Dunlap Department of Astronomy &amp; ...,"['companion', 'system', 'planet', 'mass', 'gia...","Based on the top words, I determine that the p...","['Based on the top words and general topic, I ...","['planetary systems', 'stars']",['planetary systems - planets and satellites: ...,25
1,"Volonteri, Marta","Institut d'Astrophysique de Paris, Sorbonne Un...","['hole', 'black', 'galaxy', 'mass', 'merger', ...","Based on the top words, I determine that the p...","['Based on the top words and general topic, I ...","['galaxies', 'cosmology', 'black holes']",['galaxies - galaxies: nuclei|galaxies: evolut...,73
2,"Martens, Petrus","Georgia State University, Atlanta, United Stat...","['solar', 'data', 'feature', 'module', 'magnet...","Based on the top words, I determine that the p...","['Based on the top words and general topic, I ...","['physical data and processes', 'the sun']",['physical data and processes - magnetic field...,38
3,"Martin, David","Tufts University, Department of Physics &amp; ...","['planet', 'binary', 'star', 'mass', 'circumbi...",Here is the list of topics in a Python list fo...,"['Based on the top words and general topic, I ...","['planetary systems', 'stars', 'galaxies', 'co...",['planetary systems - planets and satellites: ...,66
4,"Cordiner, Martin","-, -, Catholic University of America, Solar Sy...","['interstellar', 'observation', 'cn', 'coma', ...","Based on the top words, I determine the areas ...","['Based on the top words and general topic, I ...","['interstellar medium (ism) nebulae', 'astrono...",['ISM - molecules|ISM: lines and bands|ISM: st...,95
...,...,...,...,...,...,...,...,...
834,"Martinez, Raquel","University of California, Irvine, University o...","['star', 'mass', 'companion', 'system', 'wide'...","Based on the top words, I determine that the p...","['Based on the top words and general topic, I ...","['stars', 'planetary systems', 'interstellar m...",['stars - stars: formation|stars: planetary sy...,25
835,"Nelson, Richard","Deceased, 1. Mountain Ash Observatory, BC, Can...","['surface', 'reflectance', 'disc', 'titan', 'a...","Based on the top words, I determine the areas ...","['Based on the top words and general topic, I ...",['planetary systems'],['planetary systems - planets and satellites: ...,92
836,"Walker, Reddick","-, -, -, -, -, -, Department of Earth, Planeta...","['data', 'earth', 'reconnection', 'simulation'...","Based on the top words, I determine the areas ...","['Based on the top words and general topic, I ...","['physical data and processes', 'the sun']",['physical data and processes - magnetic recon...,84
837,"Fortenberry, Ryan","-, Department of Chemistry &amp; Biochemistry,...","['molecule', 'vibrational', 'state', 'frequenc...","Based on the provided top words, I determine t...","[""Based on the top words and general topic, I ...","['physical data and processes', 'interstellar ...",['physical data and processes - molecular data...,35


In [6]:
import ast
def format_output(output):
  lst = ast.literal_eval(output)
  format = ""
  for item in lst:
    try:
      topics = item.split(' - ')[0]
      subtopic_split = item.split(' - ')[1]
      subtopics = subtopic_split.split('|')
      format += topics+':'+str(subtopics)+'|'
    except:
      pass
  format+="Number of Papers: "
  return format

output_df["Expertise"] = output_df["Subtopics"].apply(format_output)
output_df["Expertise"] = output_df['Expertise'] + output_df['Number of Papers'].astype(str)
output_df

,Input Author,Affiliations,Combined Top Words,Topics with Explanation,Subtopics with Explanation,Topics,Subtopics,Number of Papers,Expertise
0,"Bryan, Marta",David A. Dunlap Department of Astronomy &amp; ...,"['companion', 'system', 'planet', 'mass', 'gia...","Based on the top words, I determine that the p...","['Based on the top words and general topic, I ...","['planetary systems', 'stars']",['planetary systems - planets and satellites: ...,25,planetary systems:['planets and satellites: ge...
1,"Volonteri, Marta","Institut d'Astrophysique de Paris, Sorbonne Un...","['hole', 'black', 'galaxy', 'mass', 'merger', ...","Based on the top words, I determine that the p...","['Based on the top words and general topic, I ...","['galaxies', 'cosmology', 'black holes']",['galaxies - galaxies: nuclei|galaxies: evolut...,73,"galaxies:['galaxies: nuclei', 'galaxies: evolu..."
2,"Martens, Petrus","Georgia State University, Atlanta, United Stat...","['solar', 'data', 'feature', 'module', 'magnet...","Based on the top words, I determine that the p...","['Based on the top words and general topic, I ...","['physical data and processes', 'the sun']",['physical data and processes - magnetic field...,38,physical data and processes:['magnetic fields'...
3,"Martin, David","Tufts University, Department of Physics &amp; ...","['planet', 'binary', 'star', 'mass', 'circumbi...",Here is the list of topics in a Python list fo...,"['Based on the top words and general topic, I ...","['planetary systems', 'stars', 'galaxies', 'co...",['planetary systems - planets and satellites: ...,66,planetary systems:['planets and satellites: fo...
4,"Cordiner, Martin","-, -, Catholic University of America, Solar Sy...","['interstellar', 'observation', 'cn', 'coma', ...","Based on the top words, I determine the areas ...","['Based on the top words and general topic, I ...","['interstellar medium (ism) nebulae', 'astrono...",['ISM - molecules|ISM: lines and bands|ISM: st...,95,"ISM:['molecules', 'ISM: lines and bands', 'ISM..."
...,...,...,...,...,...,...,...,...,...
834,"Martinez, Raquel","University of California, Irvine, University o...","['star', 'mass', 'companion', 'system', 'wide'...","Based on the top words, I determine that the p...","['Based on the top words and general topic, I ...","['stars', 'planetary systems', 'interstellar m...",['stars - stars: formation|stars: planetary sy...,25,"stars:['stars: formation', 'stars: planetary s..."
835,"Nelson, Richard","Deceased, 1. Mountain Ash Observatory, BC, Can...","['surface', 'reflectance', 'disc', 'titan', 'a...","Based on the top words, I determine the areas ...","['Based on the top words and general topic, I ...",['planetary systems'],['planetary systems - planets and satellites: ...,92,planetary systems:['planets and satellites: su...
836,"Walker, Reddick","-, -, -, -, -, -, Department of Earth, Planeta...","['data', 'earth', 'reconnection', 'simulation'...","Based on the top words, I determine the areas ...","['Based on the top words and general topic, I ...","['physical data and processes', 'the sun']",['physical data and processes - magnetic recon...,84,physical data and processes:['magnetic reconne...
837,"Fortenberry, Ryan","-, Department of Chemistry &amp; Biochemistry,...","['molecule', 'vibrational', 'state', 'frequenc...","Based on the provided top words, I determine t...","[""Based on the top words and general topic, I ...","['physical data and processes', 'interstellar ...",['physical data and processes - molecular data...,35,"physical data and processes:['molecular data',..."


In [8]:
file_name = '/content/drive/MyDrive/Colab Notebooks/SMS/3853-4853.csv'
sample_df = pd.read_csv(file_name)
sample_df

final_df = sample_df.merge(output_df, left_on="Name", right_on="Input Author")
final_df.rename(columns={"Expertise_x": "Labeled Expertise", "Expertise_y": "Model Expertise"}, inplace=True)
final_df

,Name,Title,LastName,Organization,Email,Labeled Expertise,Phone,MiddleName,FirstName,Notes,ID,Input Author,Affiliations,Combined Top Words,Topics with Explanation,Subtopics with Explanation,Topics,Subtopics,Number of Papers,Model Expertise
0,"Bryan, Marta",Marta,Bryan,NaN,martalbryan@berkeley.edu,"ExoPlanet, debris disks, gas giants",NaN,NaN,NaN,NaN,3352,"Bryan, Marta",David A. Dunlap Department of Astronomy &amp; ...,"['companion', 'system', 'planet', 'mass', 'gia...","Based on the top words, I determine that the p...","['Based on the top words and general topic, I ...","['planetary systems', 'stars']",['planetary systems - planets and satellites: ...,25,planetary systems:['planets and satellites: ge...
1,"Bryan, Marta",Marta,Bryan,NaN,martalbryan@berkeley.edu,"ExoPlanet, debris disks, gas giants",NaN,NaN,NaN,NaN,3352,"Bryan, Marta",David A. Dunlap Department of Astronomy &amp; ...,"['companion', 'system', 'planet', 'mass', 'gia...","Based on the top words, I determine that the p...","['Based on the top words and general topic, I ...","['planetary systems', 'stars']",['planetary systems - planets and satellites: ...,25,planetary systems:['planets and satellites: ge...
2,"Volonteri, Marta",Marta,Volonteri,Self,martav@iap.fr,NaN,734-647-4212,NaN,NaN,NaN,814,"Volonteri, Marta","Institut d'Astrophysique de Paris, Sorbonne Un...","['hole', 'black', 'galaxy', 'mass', 'merger', ...","Based on the top words, I determine that the p...","['Based on the top words and general topic, I ...","['galaxies', 'cosmology', 'black holes']",['galaxies - galaxies: nuclei|galaxies: evolut...,73,"galaxies:['galaxies: nuclei', 'galaxies: evolu..."
3,"Volonteri, Marta",Marta,Volonteri,Self,martav@iap.fr,NaN,734-647-4212,NaN,NaN,NaN,814,"Volonteri, Marta","Institut d'Astrophysique de Paris, Sorbonne Un...","['hole', 'black', 'galaxy', 'mass', 'merger', ...","Based on the top words, I determine that the p...","['Based on the top words and general topic, I ...","['galaxies', 'cosmology', 'black holes']",['galaxies - galaxies: nuclei|galaxies: evolut...,73,"galaxies:['galaxies: nuclei', 'galaxies: evolu..."
4,"Martens, Petrus",Petrus,Martens,"Georgia State University Research Foundation, ...",martens@astro.gsu.edu,NaN,404-413-6612,C,NaN,NaN,2380,"Martens, Petrus","Georgia State University, Atlanta, United Stat...","['solar', 'data', 'feature', 'module', 'magnet...","Based on the top words, I determine that the p...","['Based on the top words and general topic, I ...","['physical data and processes', 'the sun']",['physical data and processes - magnetic field...,38,physical data and processes:['magnetic fields'...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
964,"Nelson, Richard",Richard,Nelson,"Queen Mary, University of London",r.p.nelson@qmul.ac.uk,NaN,NaN,NaN,NaN,NaN,4322,"Nelson, Richard","Deceased, 1. Mountain Ash Observatory, BC, Can...","['surface', 'reflectance', 'disc', 'titan', 'a...","Based on the top words, I determine the areas ...","['Based on the top words and general topic, I ...",['planetary systems'],['planetary systems - planets and satellites: ...,92,planetary systems:['planets and satellites: su...
965,"Walker, Reddick",Reddick,Walker,UNIVERSITY OF FLORIDA,r.walker@ufl.edu,NaN,352-294-3072,R,NaN,NaN,6621,"Walker, Reddick","-, -, -, -, -, -, Department of Earth, Planeta...","['data', 'earth', 'reconnection', 'simulation'...","Based on the top words, I determine the areas ...","['Based on the top words and general topic, I ...","['physical data and processes', 'the sun']",['physical data and processes - magnetic recon...,84,physical data and processes:['magnetic reconne...
966,"Fortenberry, Ryan",Ryan,Fortenberry,University Of Mississippi,r410@olemiss.edu,NaN,662-915-1687,C,NaN,NaN,6307,"Fortenberry, Ryan","-, Department of Chemistry &amp; Biochemistry,...","['molecule', 'vibrational', 'state', 'frequenc...","Based on the provided top words, I determine t...","[""Based on the top words and general to

In [10]:
final_df.to_excel(path_stop+"final_3853-4853.xlsx", index=False)